In [1]:
import sys
sys.path.append("..")

from gnn_utils.gnn_model_utils import load_trained_gnn_model
from gnn_utils.inverse_design_utils import optimize_input_for_target

import numpy as np
import torch

In [2]:
feature_cols_dict = {
    "camb": ["AccuracyBoost", "lAccuracyBoost", "lSampleBoost"],
    "class": ["tol_background", "tol_thermo"],
    "emcee": ["n_walkers", "n_steps"]
}

graph_paths_dict = {
    "camb": "../Data/camb_graph.pt",
    "class": "../Data/class_graph.pt",
    "emcee": "../Data/emcee_graph.pt"
}

model_paths_dict = {
    "camb": "../Data/gnn_camb_model.pt",
    "class": "../Data/gnn_class_model.pt",
    "emcee": "../Data/gnn_emcee_model.pt"
}

def run_inverse_design(
    label,
    desired_error,
    feature_cols_dict,
    graph_paths_dict,
    model_paths_dict,
    num_steps=1200,
    lr=0.05,
    device=None
):
    """
    Mode 2: Given desired_error, find X that MINIMIZES predicted CPU time.
    Uses updated loss logic inside optimize_input_for_target.
    """

    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # ------------------------------------------
    # Load model + graph
    # ------------------------------------------
    model_path = model_paths_dict[label]
    graph_path = graph_paths_dict[label]

    model_loaded, data, scaler_x, scaler_y = load_trained_gnn_model(
        model_path=model_path,
        graph_path=graph_path,
        device=device
    )

    # ------------------------------------------
    # Bounds based on label
    # ------------------------------------------
    if label == "camb":
        bounds = {0:(1.0,3.0), 1:(1.0,3.0), 2:(1.0,3.0)}
        integer_indices = None

    elif label == "class":
        bounds = {0:(1e-10,1e-6), 1:(1e-6,1e-4)}
        integer_indices = None

    elif label == "emcee":
        bounds = {0:(50,500), 1:(200,5000)}
        integer_indices = [0,1]

    else:
        raise ValueError(f"Unknown label: {label}")

    # ------------------------------------------
    # Call inverse design optimizer
    # Only give desired_error (Mode 2)
    # ------------------------------------------
    X_orig, X_norm, y_pred_orig = optimize_input_for_target(
        model=model_loaded,
        scaler_x=scaler_x,
        scaler_y=scaler_y,
        desired_error_orig=desired_error,  # cpu_time auto-optimized
        feature_dim=data.x.shape[1],
        num_steps=num_steps,
        lr=lr,
        bounds_orig=bounds,
        integer_indices=integer_indices,
        device=device
    )

    # ------------------------------------------
    # Print results
    # ------------------------------------------
    print("\n=== INVERSE DESIGN: MODE 2 (Minimize CPU Time given desired error) ===")
    print(f"Desired error: {desired_error}")
    print(f"Label: {label.upper()}")

    print("\nSuggested Input Settings:")
    for name, val in zip(feature_cols_dict[label], X_orig[0]):
        if integer_indices and feature_cols_dict[label].index(name) in integer_indices:
            print(f"{name}: {int(val)}")
        else:
            print(f"{name}: {val:.6f}")

    print("\nPredicted [error, cpu_time]:")
    print(y_pred_orig[0])

    return X_orig, y_pred_orig


In [3]:
target_error = 5e-6
X_orig, y_pred_orig = run_inverse_design("camb", desired_error=target_error,
                       feature_cols_dict=feature_cols_dict,
                       graph_paths_dict=graph_paths_dict,
                       model_paths_dict=model_paths_dict,
                        num_steps=1000)


Loaded trained GNN model from: ../Data/gnn_camb_model.pt
Loaded graph from: ../Data/camb_graph.pt
[inverse] step 0, loss=-0.013675, error=0.2627, cpu=-0.1368
[inverse] step 100, loss=-0.123376, error=0.5406, cpu=-1.2338
[inverse] step 200, loss=-0.123937, error=0.6096, cpu=-1.2395
[inverse] step 300, loss=-0.123971, error=0.6094, cpu=-1.2398
[inverse] step 400, loss=-0.123953, error=0.6157, cpu=-1.2403
[inverse] step 500, loss=-0.123970, error=0.6116, cpu=-1.2399
[inverse] step 600, loss=-0.124002, error=0.6156, cpu=-1.2408
[inverse] step 700, loss=-0.139348, error=-0.8743, cpu=-1.3935
[inverse] step 800, loss=-0.139354, error=-0.8739, cpu=-1.3935
[inverse] step 900, loss=-0.139354, error=-0.8739, cpu=-1.3935

=== INVERSE DESIGN: MODE 2 (Minimize CPU Time given desired error) ===
Desired error: 5e-06
Label: CAMB

Suggested Input Settings:
AccuracyBoost: 1.000000
lAccuracyBoost: 1.833328
lSampleBoost: 1.000000

Predicted [error, cpu_time]:
[-2.7585791e-05  7.1642532e+00]


In [4]:
run_inverse_design("class", desired_error=1e-4,
                   feature_cols_dict=feature_cols_dict,
                   graph_paths_dict=graph_paths_dict,
                   model_paths_dict=model_paths_dict)

Loaded trained GNN model from: ../Data/gnn_class_model.pt
Loaded graph from: ../Data/class_graph.pt
[inverse] step 0, loss=33589228.000000, error=-0.2587, cpu=0.5033
[inverse] step 100, loss=33583220.000000, error=-0.7776, cpu=-0.7119
[inverse] step 200, loss=33583212.000000, error=-0.7776, cpu=-0.7367
[inverse] step 300, loss=33583232.000000, error=-0.7766, cpu=-0.7131
[inverse] step 400, loss=33583212.000000, error=-0.7780, cpu=-0.7351
[inverse] step 500, loss=33583212.000000, error=-0.7778, cpu=-0.7322
[inverse] step 600, loss=33583212.000000, error=-0.7776, cpu=-0.7246
[inverse] step 700, loss=33583220.000000, error=-0.7773, cpu=-0.7248
[inverse] step 800, loss=33583220.000000, error=-0.7776, cpu=-0.7333
[inverse] step 900, loss=33583212.000000, error=-0.7778, cpu=-0.7413
[inverse] step 1000, loss=33583212.000000, error=-0.7778, cpu=-0.7204
[inverse] step 1100, loss=33583212.000000, error=-0.7779, cpu=-0.7376

=== INVERSE DESIGN: MODE 2 (Minimize CPU Time given desired error) ===
D

(array([[5.002308e-07, 6.714357e-05]], dtype=float32),
 array([[0.00182583, 1.716568  ]], dtype=float32))

In [5]:
run_inverse_design("emcee", desired_error=-6,
                   feature_cols_dict=feature_cols_dict,
                   graph_paths_dict=graph_paths_dict,
                   model_paths_dict=model_paths_dict)

Loaded trained GNN model from: ../Data/gnn_emcee_model.pt
Loaded graph from: ../Data/emcee_graph.pt
[inverse] step 0, loss=33804.296875, error=0.6085, cpu=0.0017
[inverse] step 100, loss=33593.386719, error=0.0339, cpu=0.2839
[inverse] step 200, loss=33587.722656, error=0.0185, cpu=0.3108
[inverse] step 300, loss=33588.125000, error=0.0196, cpu=0.3327
[inverse] step 400, loss=33595.496094, error=0.0397, cpu=0.3920
[inverse] step 500, loss=33595.058594, error=0.0385, cpu=0.2788
[inverse] step 600, loss=33584.160156, error=0.0088, cpu=0.2980
[inverse] step 700, loss=33586.394531, error=0.0149, cpu=0.3166
[inverse] step 800, loss=33584.273438, error=0.0091, cpu=0.3027
[inverse] step 900, loss=33595.835938, error=0.0406, cpu=0.3000
[inverse] step 1000, loss=33589.089844, error=0.0222, cpu=0.3155
[inverse] step 1100, loss=33587.937500, error=0.0191, cpu=0.3124

=== INVERSE DESIGN: MODE 2 (Minimize CPU Time given desired error) ===
Desired error: -6
Label: EMCEE

Suggested Input Settings:
n_

(array([[ 296., 2864.]], dtype=float32),
 array([[-0.01503748, 15.033895  ]], dtype=float32))